In [1]:
%load_ext autoreload
%autoreload 2

### Carla tracks explorative data analysis

Main task of this notebook is to get a sense for the carla data.

In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.spatial import distance
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, LineString
import pandas as pd
#from src.generate_traffic_data import clean_carla, spawn_scene, run_scene
import carla
import os
os.getcwd()
from datetime import datetime
import time
import random
from tqdm import tqdm
import networkx as nx
import numpy as np


os.chdir('../..')
os.getcwd()

from graph_creator.MapGraph import MapGraph
from graph_creator.ActorGraph import ActorGraph

In [3]:
# client = carla.Client('localhost', 2000)
# client.set_timeout(10.0)
# world = client.get_world()
# world_map = world.get_map()

### Read map graph and tracks and try to create actor graph.

In [4]:
g_map = MapGraph()
#g = give_it_a_try.create_from_carla_map(world_map)
#g.store_graph_to_file('test.pickle')
#give_it_a_try.visualize_graph()
#g.visualize_graph()
# type(g)
# g.graph.nodes

In [5]:
g_map.read_graph_from_file('/home/tmuehlen/repos/graph_coverage/carla/data/scene_0_2025-05-04_map_graph.pickle')

In [6]:
pd.DataFrame(g_map.graph.nodes(data=True))
g_map.graph.nodes(data=True)

NodeDataView({'447_1': {'node_info': NodeInfo(lane_id='447_1', is_intersection=True, length=11.231738284230232, lane_polygon=<POLYGON ((3.402 185.554, -3.455 179.24, -3.457 183.24, 3.4 189.554, 3.402 1...>, left_boundary=None, right_boundary=None)}, '1_-1': {'node_info': NodeInfo(lane_id='1_-1', is_intersection=False, length=63.42519770562649, lane_polygon=<POLYGON ((-1.456 181.239, -1.36 117.564, -5.36 117.565, -5.456 181.24, -1.4...>, left_boundary=None, right_boundary=None)}, '437_1': {'node_info': NodeInfo(lane_id='437_1', is_intersection=True, length=17.67089468613267, lane_polygon=<POLYGON ((-9.456 181.241, 1.399 191.554, 5.399 191.553, -5.456 181.24, -9.4...>, left_boundary=None, right_boundary=None)}, '4_-1': {'node_info': NodeInfo(lane_id='4_-1', is_intersection=False, length=31.050002921372652, lane_polygon=<POLYGON ((3.398 193.554, 34.698 193.568, 34.7 189.568, 3.4 189.554, 3.398 1...>, left_boundary=None, right_boundary=None)}, '433_1': {'node_info': NodeInfo(lane_id='433_1

In [7]:
pd.DataFrame(g_map.graph.edges(data=True))


,0,1,2
0,447_1,1_-1,{'edge_type': 'following'}
1,1_-1,16_-1,{'edge_type': 'following'}
2,1_-1,1_1,{'edge_type': 'opposite'}
3,437_1,4_-1,{'edge_type': 'following'}
4,4_-1,232_-1,{'edge_type': 'following'}
...,...,...,...
171,245_1,6_1,{'edge_type': 'following'}
172,157_-1,9_-1,{'edge_type': 'following'}
173,99_-1,7_1,{'edge_type': 'following'}
174,277_1,14_1,{'edge_type': 'following'}


In [8]:
#tracks = pd.read_parquet('/home/tmuehlen/repos/graph_coverage/carla/data/scene_0_2025-03-23_tracks.parquet')
tracks = pd.read_parquet('/home/tmuehlen/repos/graph_coverage/carla/data/scene_0_2025-05-04_tracks.parquet')

tracks['road_lane_id'] = tracks.road_id.astype(str) + '_' + tracks.lane_id.astype(str)
print(tracks.shape)

timestamps = tracks.timestamp.unique().tolist()
actors = tracks.actor_id.unique().tolist()
tracks.head(2).T

(7000, 24)


,0,1
actor_id,104,103
actor_type,vehicle.nissan.micra,vehicle.volkswagen.t2
actor_speed_xyz,"[0.0, 0.0, -0.9800000786781311]","[0.0, 0.0, -0.9800000786781311]"
actor_acceleration_xyz,"[0.0, 0.0, -9.800002098083496]","[0.0, 0.0, -9.800002098083496]"
actor_location_xyz,"[-7.529999732971191, 270.72998046875, 0.446100...","[55.410037994384766, 105.54998779296875, 0.446..."
actor_bbox,"[[-1.8166879415512085, -0.9215882420539856, 0....","[[-2.2388854026794434, -1.035194754600525, -0...."
distance_till_lane_end,23.720609,50.367339
distance_from_lane_start,71.002709,125.747272
t_per_lane_id,0.121932,-0.155708
lane_id,1,-1


In [9]:
tracks.timestamp.nunique()

200

In [10]:
ag = ActorGraph()
ag_carla = ag.from_carla_scenario(tracks, g_map, max_distance_lead_veh_m=120)
ag_carla.actor_graphs.keys()

dict_keys([6.681372907827608])

In [13]:
#ag._create_track_data_carla(tracks)
tracks.actor_type.value_counts()


actor_type
vehicle.volkswagen.t2                400
vehicle.yamaha.yzf                   400
vehicle.citroen.c3                   400
vehicle.diamondback.century          400
vehicle.tesla.model3                 400
vehicle.kawasaki.ninja               400
vehicle.mini.cooper_s_2021           400
vehicle.lincoln.mkz_2017             400
vehicle.jeep.wrangler_rubicon        400
vehicle.nissan.micra                 200
vehicle.bh.crossbike                 200
vehicle.dodge.charger_police_2020    200
vehicle.carlamotors.firetruck        200
vehicle.carlamotors.carlacola        200
vehicle.nissan.patrol_2021           200
vehicle.lincoln.mkz_2020             200
vehicle.ford.mustang                 200
vehicle.carlamotors.european_hgv     200
vehicle.chevrolet.impala             200
vehicle.vespa.zx125                  200
vehicle.tesla.cybertruck             200
vehicle.ford.ambulance               200
vehicle.audi.etron                   200
vehicle.toyota.prius                 200
vehic

In [ ]:
tracks.timestamp.value_counts()

In [ ]:
# clean up the graphs and components:

#ag_carla.actor_components[t_idx][i].size()
keys = list(ag_carla.actor_graphs.keys())
for  key in keys:
    ag_carla.actor_components[key] = [ag_carla.actor_components[key][i] for i in range(len(ag_carla.actor_components[key])) if ag_carla.actor_components[key][i].size() > 1]


In [ ]:
t_idx = 24
ag_carla.visualize_actor_graph(t_idx = t_idx, comp_idx = 0, use_map_pos=True, node_size = 100, graph_or_component='graph')
for i in range(len(ag_carla.actor_components[t_idx])):
    if ag_carla.actor_components[t_idx][i].size() > 1:
        ag_carla.visualize_actor_graph(t_idx = t_idx, comp_idx = i, use_map_pos=True, node_size = 100, graph_or_component='component')


### Start comparing graphs:

In [ ]:
t_idx = 24
comp_idx = 2
G1 = ag_carla.actor_components[t_idx][comp_idx]
ag_carla.visualize_actor_graph(t_idx = t_idx, comp_idx = comp_idx, use_map_pos=True, node_size = 100, graph_or_component='component')
t_idx = 2
comp_idx = 1
G2 = ag_carla.actor_components[t_idx][comp_idx]
ag_carla.visualize_actor_graph(t_idx = t_idx, comp_idx = comp_idx, use_map_pos=True, node_size = 100, graph_or_component='component')


In [ ]:
# node statistics:

print('degrees:', G1.degree())



In [ ]:
# graph statistics:
print('density: ', nx.density(G1))
print('diameter: ', nx.diameter(G1))
#print('clustering: ', nx.average_clustering(G))


### graph comparison:

In [ ]:
print('isomorphic:', nx.is_isomorphic(G1, G2)) # actually not meaningful, not what I understood
print('graph edit distsance:', nx.graph_edit_distance(G1, G2))
print('graph edit distsance similarity:', nx.algorithms.similarity.graph_edit_distance(G1, G2))


In [ ]:
GM = nx.algorithms.isomorphism.DiGraphMatcher(G1, G2)
print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
GM = nx.algorithms.isomorphism.DiGraphMatcher(
    G1, G2,
    node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
    edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
G1.nodes(data=True)

### Define coverage categories in graphs:

In [ ]:
# most simple: follow lead vehicle:
G = nx.MultiDiGraph()
G.add_node(1, actor_type = 'vehicle')
G.add_node(2, actor_type='vehicle')
G.add_edge(1, 2, edge_type="leading_vehicle")
G.add_edge(2, 1, edge_type="following_lead")

In [ ]:
print(G.nodes(data=True))
print(G.edges(data=True))
print('')
print(G1.nodes(data=True))
print(G1.edges(data=True))
print('')
print(G2.nodes(data=True))
print(G2.edges(data=True))



In [ ]:
GM = nx.algorithms.isomorphism.DiGraphMatcher(
    G2, G,
    node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
    edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
GM = nx.algorithms.isomorphism.DiGraphMatcher(
    G1, G,
    #node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
    edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
GM = nx.algorithms.isomorphism.MultiDiGraphMatcher(
    G1, G,
    #node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
    edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
)

print('isomorphic:', GM.is_isomorphic())
print('subgraph isomorphic:', GM.subgraph_is_isomorphic())

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import isomorphism

# Step 1: Create the host graph G (Directed)
G = nx.DiGraph()
G.add_weighted_edges_from([
    ('A', 'B', 1),
    ('B', 'C', 2),
    ('C', 'D', 3),
    ('A', 'D', 4),
    ('D', 'E', 2),
    ('E', 'F', 1),
])

# Step 2: Create the pattern graph H (Subgraph we're looking for)
H = nx.DiGraph()
H.add_weighted_edges_from([
    ('X', 'Y', 2),
    ('Y', 'Z', 3),
])

# Step 3: Define the edge matcher based on 'weight'
edge_matcher = isomorphism.categorical_edge_match('weight', 0)

# Step 4: Create GraphMatcher with edge attributes
GM = isomorphism.DiGraphMatcher(G, H, edge_match=edge_matcher)

# Step 5: Visualization

# Draw the host graph G
pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(8, 6))
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=800, arrows=True)

# Label edge weights
edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Highlight matched subgraphs
if GM.subgraph_is_isomorphic():
    print("✅ H is a subgraph of G!\n📌 Matching mappings:")
    for i, mapping in enumerate(GM.subgraph_isomorphisms_iter(), 1):
        print(f"Match {i}: {mapping}")
        
        # Extract edges from the match
        matched_edges = []
        #reverse_map = {v: k for k, v in mapping.items()}
        reverse_mapping = {v: k for k, v in mapping.items()}
        for u, v in H.edges():
            if u in reverse_mapping and v in reverse_mapping:
                source = reverse_mapping[u]
                target = reverse_mapping[v]
                matched_edges.append((source, target))
        
        # Draw matched edges in red
        nx.draw_networkx_edges(G, pos, edgelist=matched_edges, edge_color='red', width=3)

else:
    print("❌ H is NOT a subgraph of G.")

plt.title("Subgraph Isomorphism with Edge Matching")
plt.show()
